<a href="https://colab.research.google.com/github/ameerfayiz/ML-notebooks/blob/main/Zero_shot_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import json
from transformers import pipeline
import time
import torch

torch.cuda.is_available()

labels = ['travel', 'friends','technology','family' ,'algorithms','cooking', 'job' , 'dancing','health','happiness','activity','social' , 'exploration']

# classifier = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")                  # 1.63 Gb 0.4384075531592736 per label
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")      # 558  Mb 0.313440519112807 per label
# classifier = pipeline("zero-shot-classification", model="BaptisteDoyen/camembert-base-xnli")            # 443 Mb 0.12476613338177021 per label
# classifier = pipeline("zero-shot-classification", model="typeform/distilbert-base-uncased-mnli")        # 268 Mb 0.05824783215155968 per label
# classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-1")                # 890 Mb 0.20728054413428673 per label 
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli") # 369 Mb 0.3029702296623817 per label 
# classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-base")            # 738 Mb 0.30590915496532733 per label
# classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-v3-small")           # 568 Mb 0.15964309252225434 per label
# classifier = pipeline("zero-shot-classification", model="cross-encoder/nli-MiniLM2-L6-H768")            # 329 Mb 0.055275693306556115 per label
classifier = pipeline("zero-shot-classification", model="typeform/mobilebert-uncased-mnli")            # 99 Mb 0.03554045275637978 per label



print("model loaded!")


def classify(prompt,multi_label=True):
    begin = time.time()
    for i in range(10):
      result = classifier(prompt, labels,multi_label=multi_label)
    end = time.time()
    print(json.dumps(result,indent="  "))
    # total time taken
    gap = (end - begin)/10
    print(f"Total runtime of the program is {gap} is, {gap/len(labels)} per label")


classify("To avoid plagiarism issues, please refer to the code on the Google Colab")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


model loaded!
{
  "sequence": "To avoid plagiarism issues, please refer to the code on the Google Colab",
  "labels": [
    "technology",
    "social",
    "activity",
    "algorithms",
    "exploration",
    "travel",
    "health",
    "family",
    "job",
    "cooking",
    "friends",
    "dancing",
    "happiness"
  ],
  "scores": [
    0.5289000272750854,
    0.25394248962402344,
    0.22862575948238373,
    0.21224497258663177,
    0.10195177793502808,
    0.10091692954301834,
    0.06666842103004456,
    0.06201644986867905,
    0.041554179042577744,
    0.031228413805365562,
    0.013975279405713081,
    0.00899865385144949,
    0.0058839586563408375
  ]
}
Total runtime of the program is 0.5481183290481567 is, 0.04216294838831975 per label


In [42]:
classify("Got up early at 04:40 to drive my mom to the airport Went back to sleep til 09:00")

{
  "sequence": "Got up early at 04:40 to drive my mom to the airport Went back to sleep til 09:00",
  "labels": [
    "social",
    "activity",
    "family",
    "health",
    "travel",
    "exploration",
    "algorithms",
    "job",
    "happiness",
    "cooking",
    "technology",
    "friends",
    "dancing"
  ],
  "scores": [
    0.912135660648346,
    0.9059642553329468,
    0.6507245898246765,
    0.5867769122123718,
    0.5659762024879456,
    0.44396260380744934,
    0.19942346215248108,
    0.16412216424942017,
    0.1610601544380188,
    0.09640653431415558,
    0.04231642559170723,
    0.02215701900422573,
    0.021515414118766785
  ]
}
Total runtime of the program is 0.5576441287994385 is, 0.04289570221534142 per label


In [50]:
classify("Watched Netflix series Kaleidiscope")

{
  "sequence": "Watched Netflix series Kaleidiscope",
  "labels": [
    "activity",
    "exploration",
    "entertainment",
    "travel",
    "social",
    "family",
    "algorithms",
    "technology",
    "party",
    "dancing",
    "job",
    "games",
    "happiness",
    "friends",
    "business",
    "health",
    "cooking",
    "shopping",
    "groceries"
  ],
  "scores": [
    0.901907205581665,
    0.8632742166519165,
    0.7707504034042358,
    0.5981687307357788,
    0.5333359837532043,
    0.2073981612920761,
    0.14227071404457092,
    0.08864321559667587,
    0.07894536852836609,
    0.0693483054637909,
    0.05788259953260422,
    0.0480632483959198,
    0.04455789923667908,
    0.043071016669273376,
    0.04021870717406273,
    0.03262107074260712,
    0.018449250608682632,
    0.009433865547180176,
    0.004163410980254412
  ]
}
Total runtime of the program is 0.6752686023712158 is, 0.03554045275637978 per label
